In [107]:
# Iris Flower Predictor

In [108]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style()

# Load Data

In [109]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"

train_dataset_address = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)
test_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"

test_dataset_address = tf.keras.utils.get_file(fname=os.path.basename(test_dataset_url),
                                           origin=test_dataset_url)

print("Local copy of the Training dataset file: {}".format(train_dataset_address))
print("Local copy of the Test dataset file: {}".format(test_dataset_address))

Local copy of the Training dataset file: C:\Users\LENOVO\.keras\datasets\iris_training.csv
Local copy of the Test dataset file: C:\Users\LENOVO\.keras\datasets\iris_test.csv


In [110]:
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

species = ['Setosa', 'Versicolor', 'Virginica']

In [111]:
x_train = pd.read_csv(train_dataset_address,names=column_names,header=0)
x_test =  pd.read_csv(test_dataset_address,names=column_names,header=0)

# x_train = x_train.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
# x_test = x_test.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

y_train = x_train.pop('species')
y_test = x_test.pop('species')

# Input Function 

In [112]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [113]:
# Feature columns describe how to use the input.
feature_columns = []
for key in x_train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))
print(feature_columns)

[NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Model Creation

In [114]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LENOVO\\AppData\\Local\\Temp\\tmp3uez1qn2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [115]:
classifier.train(
    input_fn = lambda: input_fn(x_train, y_train, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\LENOVO\AppData\Local\Temp\tmp3uez1qn2\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1818593, step = 0
INFO:tensorflow:global_step/sec: 196.067
INFO:tensorflow:loss = 1.0421673, step = 100 (0.510 sec)
INFO:tensorflow:global_step/sec: 172.399
INFO:tensorflow:loss = 1.0036216, step = 200 (0.580 sec)
INFO:tensorflow:global_step/sec: 144.982
INFO:tensorflow:loss = 0.98209715, step = 300 (0.690 sec)
INFO:tensorflow:global_step/sec: 152.764
INFO:tensorflow:loss = 0.96219987, step = 400 (0.660 sec)
INFO:tensorflow:global_step/sec: 210.498
INFO:tensorflow:loss = 0.941244

In [116]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(x_test, y_test, training=False))

print('\n Accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-08-12T20:02:28
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\LENOVO\AppData\Local\Temp\tmp3uez1qn2\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.36395s
INFO:tensorflow:Finished evaluation at 2021-08-12-20:02:29
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.9, average_loss = 0.62192446, global_step = 5000, loss = 0.62192446
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\LENOVO\AppData\Local\Temp\tmp3uez1qn2\model.ckpt-5000

 Accuracy: 0.900

